In [1]:
# %load singlestage.py

import torch
import torch.nn as nn

class SimplePnPNet(nn.Module):
    def __init__(self, nIn):
        super(SimplePnPNet, self).__init__()

        self.conv1 = torch.nn.Conv1d(nIn, 128, 1)
        self.conv2 = torch.nn.Conv1d(128, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 128, 1)

        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc_qt = nn.Linear(256, 7)
        self.act = nn.LeakyReLU(0.1, inplace=True)

    def forward(self, x):
        batch_size = x.size(0)
        data_size = x.size(2)  # number of correspondences

        print(x.shape)
        x = self.act(self.conv1(x))
        print(x.shape)
        x = self.act(self.conv2(x))
        print(x.shape)
        x = self.conv3(x)
        print(x.shape)

        x = x.view(batch_size, 128, -1, 8)
        print(x.shape)
        x = torch.max(x, dim=2, keepdim=True)[0]
        print(x.shape)
        # x = torch.mean(x, dim=2, keepdim=True)

        x = x.view(batch_size, 1024)
        print(x.shape)
        # 
        x = self.act(self.fc1(x))
        x = self.act(self.fc2(x))
        # 
        qt = self.fc_qt(x)
        return qt

In [2]:
model = SimplePnPNet(4)

In [3]:
model

SimplePnPNet(
  (conv1): Conv1d(4, 128, kernel_size=(1,), stride=(1,))
  (conv2): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
  (conv3): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc_qt): Linear(in_features=256, out_features=7, bias=True)
  (act): LeakyReLU(negative_slope=0.1, inplace)
)

In [8]:
input = torch.zeros(1, 4, 1280)

In [9]:
model(input)

torch.Size([1, 4, 1280])
torch.Size([1, 128, 1280])
torch.Size([1, 128, 1280])
torch.Size([1, 128, 1280])
torch.Size([1, 128, 160, 8])
torch.Size([1, 128, 1, 8])
torch.Size([1, 1024])


tensor([[ 0.0395, -0.0340, -0.0216,  0.0271,  0.0098, -0.0348,  0.0147]],
       grad_fn=<AddmmBackward>)

In [9]:
model.fc_qt.weight

Parameter containing:
tensor([[-0.0530,  0.0611,  0.0340,  ..., -0.0058,  0.0491, -0.0478],
        [ 0.0333,  0.0041,  0.0293,  ..., -0.0208,  0.0133,  0.0189],
        [ 0.0555,  0.0571, -0.0139,  ...,  0.0545, -0.0606, -0.0331],
        ...,
        [ 0.0476,  0.0001,  0.0314,  ...,  0.0538,  0.0504,  0.0020],
        [-0.0445, -0.0077, -0.0151,  ..., -0.0078,  0.0543, -0.0323],
        [ 0.0125,  0.0466, -0.0622,  ...,  0.0593, -0.0308, -0.0354]],
       requires_grad=True)